In [ ]:
#import everything here
import numpy as np
import pandas as pd

Calculating the Simple moving average(SMA) of an equity price over a period (Middle Band)
Upper/Lower Bands = 2 S.D. from the middle band

In [ ]:
#Function calculating the SMA 
